# Extract data from SQLITE file

ETL is the most frequent process for data engineers, DE needs to integrate data from various heterogeneous sources for data analysts and data scientist.

This practice is under a context where DE needs to get data from a SQLITE file and then import it into SQL Server.

In [30]:
import sqlite3
import pyodbc # connect SQL Server

### Step 1 Extract tables and columns with sqlite 3

In [31]:
# Connect to the SQLite database file
conn = sqlite3.connect('D:/Learn_DS/Git/python-learning/ETL/data/database.sqlite')

In [32]:
# Create a cursor object
cursor = conn.cursor()

# Retrieve the table names from the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_names = cursor.fetchall()
table_names

[('reviews',),
 ('artists',),
 ('genres',),
 ('labels',),
 ('years',),
 ('content',)]

In [33]:
# Iterate over each table
for table_name in table_names:
    table_name = table_name[0]  # Extract the table name from the result tuple
    
    # Execute the PRAGMA statement to get column information
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    
    # Dictionary to store table data
    table_data = {}

    # Extract the column names
    column_names = [column[1] for column in columns]
    column_datatypes = [column[2] for column in columns]

    # Store the column names and datatypes in the dictionary
    table_data[table_name] = {
         'column_names': column_names,
         'column_datatypes': column_datatypes
    }

    # Print the column names
    print(f"Table: {table_name}")
    print("Columns:")
    for column_name in column_names:
         print(column_name)
    print("DataTypes:")
    for column_datatype in column_datatypes:
         print(column_datatype)

    print()


Table: reviews
Columns:
reviewid
title
artist
url
score
best_new_music
author
author_type
pub_date
pub_weekday
pub_day
pub_month
pub_year
DataTypes:
INTEGER
TEXT
TEXT
TEXT
REAL
INTEGER
TEXT
TEXT
TEXT
INTEGER
INTEGER
INTEGER
INTEGER

Table: artists
Columns:
reviewid
artist
DataTypes:
INTEGER
TEXT

Table: genres
Columns:
reviewid
genre
DataTypes:
INTEGER
TEXT

Table: labels
Columns:
reviewid
label
DataTypes:
INTEGER
TEXT

Table: years
Columns:
reviewid
year
DataTypes:
INTEGER
INTEGER

Table: content
Columns:
reviewid
content
DataTypes:
INTEGER
TEXT



### Step 2 Connect SQL Server then import data

In [34]:
# Establish a connection
conn_sqlserver = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-FIS40MH\MSSQLSERVER01;"
    "DATABASE_NAME=Project;"
    "Trusted_Connection=yes;"
     )

# Create a cursor object for SQL Server
cursor_sqlserver = conn_sqlserver.cursor()

In [35]:
# Iterate over each table
for table_name, table_data in table_data.items():
    #Extract column names and datatypes
    column_names = table_data['column_names']
    column_datatypes = table_data['column_datatypes']
    
    # Create the table in SQL Server
    create_table_query = f"CREATE TABLE {table_name} ("
    create_table_query += ', '.join([f"{column_name} {column_datatype}" for column_name, column_datatype in zip(column_names, column_datatypes)])
    create_table_query += ")"
    
    cursor_sqlserver.execute(create_table_query)

    # Insert data into the table
    insert_query = f"INSERT INTO {table_name} VALUES ({', '.join(['?'] * len(column_names))})"
    cursor_sqlserver.executemany(insert_query, cursor.execute(f"SELECT * FROM {table_name}").fetchall())

In [ ]:
# Close the SQLite connection
conn.close()

In [36]:
# Commit and close SQL Server connection
conn_sqlserver.commit()
conn_sqlserver.close()
